In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
import matplotlib.pylab as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


In [44]:
#7-1.a 범주형 예측 변수를 2개의 이진 변수로 변환, 3개의 이진 변수로 변환

potCos = pd.Series(['Stats', 'Others'])
dueT = pd.Series([1.0, 1.1])
course = pd.Series(['notTaken', 'taken'])
df = pd.DataFrame({'potCus':potCos, 'dueT':dueT, 'course':course})

newCus = pd.DataFrame([{'potCus':'IT', 'dueT':1.0, 'course':''}])
new_df = pd.concat([df, newCus], ignore_index=True)
new_df['course'] = new_df['course'].astype('category')


df_two = new_df.drop(['course'], axis=1)


df_two = pd.get_dummies(df_two, prefix_sep='_', drop_first=True)
df_two

df_two['potCus_Others'] = [1.0 if x == True else 0.0 for x in df_two['potCus_Others'].values]
df_two['potCus_Stats'] = [1.0 if x == True else 0.0 for x in df_two['potCus_Stats'].values]
df_two

,dueT,potCus_Others,potCus_Stats
0,1.0,0.0,1.0
1,1.1,1.0,0.0
2,1.0,0.0,0.0


In [45]:
potCus_IT = pd.Series([False, False, True])
df_three = pd.concat([df_two, potCus_IT], axis=1)
df_three.rename(columns={0:'potCus_IT'}, inplace=True)

df_three['potCus_IT'] = df_three['potCus_IT'].apply(lambda x: 1 if x == True else 0)


In [46]:
# 7-1 b.c 유클리드 거리 계산, 2 또는 3개의 가변수를 사용 했을때 차이는 없다

distances = []
def getD(ru, nu):
    for i in range(int(len(ru))):
        distance=0
        for j in range(int(len(ru.columns))):
            distance += (ru.iloc[i,j]-nu[j])**2
        distance = np.sqrt(distance)
        distances.append(distance)
    return distances

dr = df_two.iloc[0:2]
dn = df_two.iloc[2]
dist2 = getD(dr, dn)

dr = df_three.iloc[0:2]
dn = df_three.iloc[2]
dist3 = getD(dr, dn)

print(dist2)
print(dist3)

[1.0, 1.004987562112089, 1.4142135623730951, 1.4177446878757824]
[1.0, 1.004987562112089, 1.4142135623730951, 1.4177446878757824]


C:\Users\USER\AppData\Local\Temp\ipykernel_17400\803775410.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  distance += (ru.iloc[i,j]-nu[j])**2


In [47]:
# 7-2 a. k=1일때 k-NN 분류를 수행한다. 범주형 예측변수 가변수 변환하였고 새로운 고객은 제안된 개인 대출 제의를 받아들이지 않을 것이다

PLA = pd.read_csv("D:/dmba/UniversalBank.csv")
PLA.drop(columns=['ID', 'ZIP Code'], inplace=True)





In [48]:
PLA['Education'] = PLA['Education'].astype('category')
PLA.Education.cat.categories

PLA.columns = [c.replace(' ', '_') for c in PLA.columns]
bank_df = pd.get_dummies(PLA, prefix_sep='_', drop_first=True)
bank_df

,Age,Experience,Income,Family,CCAvg,Mortgage,Personal_Loan,Securities_Account,CD_Account,Online,CreditCard,Education_2,Education_3
0,25,1,49,4,1.6,0,0,1,0,0,0,False,False
1,45,19,34,3,1.5,0,0,1,0,0,0,False,False
2,39,15,11,1,1.0,0,0,0,0,0,0,False,False
3,35,9,100,1,2.7,0,0,0,0,0,0,True,False
4,35,8,45,4,1.0,0,0,0,0,0,1,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,29,3,40,1,1.9,0,0,0,0,1,0,False,True
4996,30,4,15,4,0.4,85,0,0,0,1,0,False,False
4997,63,39,24,2,0.3,0,0,0,0,0,0,False,True
4998,65,40,49,3,0.5,0,0,0,0,1,0,True,False


In [49]:
bank_df['Education_2'] = [1.0 if x == True else 0.0 for x in bank_df['Education_2'].values]
bank_df['Education_3'] = [1.0 if x == True else 0.0 for x in bank_df['Education_3'].values]

X = bank_df.drop(columns=['Personal_Loan'])
y = bank_df['Personal_Loan']

train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.4, random_state=1)

std_scale = StandardScaler()
std_scale.fit(train_X)
X_tn_std = std_scale.transform(train_X)
X_te_std = std_scale.transform(valid_X)

X_tn_std = pd.DataFrame(X_tn_std, columns=['Age', 'Experience', 'Income', 'Family', 'CCAvg', 'Mortgage', 'Securities_Account', 'CD_Account', 'Online', 'Credit_Card','Education_2', 'Education_3'])
X_te_std = pd.DataFrame(X_te_std, columns=['Age', 'Experience', 'Income', 'Family', 'CCAvg', 'Mortgage', 'Securities_Account', 'CD_Account', 'Online', 'Credit_Card','Education_2', 'Education_3'])

knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_tn_std, train_y)
pred = knn.predict(X_te_std)
predprob = knn.predict_proba(X_te_std)
newCust = pd.DataFrame([{'Age':40, 'Experience':10, 'Income':84, 'Family':2, 'CCAvg':2,  
                    'Mortgage':0, 'Securities_Account':0, 'CD_Account':0, 'Online':1, 'Credit_Card':1,'Education_2':1, 'Education_3':0,}])

newCust_df = pd.DataFrame(std_scale.fit_transform(newCust), columns=['Age', 'Experience', 'Income', 'Family', 'CCAvg', 'Mortgage', 'Securities_Account', 'CD_Account', 'Online', 'Credit_Card','Education_2', 'Education_3'])
distances, indices = knn.kneighbors(newCust_df)
print(distances)
print(indices)
print(y[indices[0][0]])

[[1.6275841]]
[[870]]
0


In [50]:
# 7-2 b. 균형을 잡아주는 k 값은 9이다. 
results = []
for k in range(1, 15):
    knn = KNeighborsClassifier(n_neighbors=k).fit(train_X, train_y)
    results.append({
        'k':k, 
        'accuracy_score':accuracy_score(valid_y, knn.predict(valid_X))
    })

results = pd.DataFrame(results)
print(results)
print(results[results['accuracy_score']==results['accuracy_score'].max()])

     k  accuracy_score
0    1          0.8970
1    2          0.9035
2    3          0.9035
3    4          0.9065
4    5          0.9060
5    6          0.9050
6    7          0.9035
7    8          0.9025
8    9          0.9080
9   10          0.9040
10  11          0.9025
11  12          0.9065
12  13          0.9045
13  14          0.9060
   k  accuracy_score
8  9           0.908


In [51]:
# 7-2 c. d. 최적의 k 9를 이용해 고객을 분류하고 혼동 행렬 확인
knn = KNeighborsClassifier(n_neighbors=9)
knn.fit(X_tn_std, train_y)
pred = knn.predict(X_te_std)
predprob = knn.predict_proba(X_te_std)
newCust = pd.DataFrame([{'Age':40, 'Experience':10, 'Income':84, 'Family':2, 'CCAvg':2,  
                    'Mortgage':0, 'Securities_Account':0, 'CD_Account':0, 'Online':1, 'Credit_Card':1,'Education_2':1, 'Education_3':0,}])

newCust_df = pd.DataFrame(std_scale.fit_transform(newCust), columns=['Age', 'Experience', 'Income', 'Family', 'CCAvg', 'Mortgage', 'Securities_Account', 'CD_Account', 'Online', 'Credit_Card','Education_2', 'Education_3'])
distances, indices = knn.kneighbors(newCust_df)
print(distances)
print(indices)
print(y[indices[0][0]])

conf_matrix = confusion_matrix(valid_y, pred)
print(conf_matrix)

[[1.6275841  1.6563556  1.68332808 1.73319934 1.737005   1.75332074
  1.77416809 1.78005071 1.78670916]]
[[870 341 365 887 201  90 472 677 717]]
0
[[1806    1]
 [ 102   91]]


In [52]:

#7-2 e. 검증 데이터를 통해 선택된 모델은 일반화 관점에서 과대평가 될 수 있으므로 테스트 데이터를 통해 모델을 재평가 할 수 있습니다. 

std_scale = preprocessing.StandardScaler()
std_scale.fit_transform(bank_df.drop(columns=['Personal_Loan']))

train, test = train_test_split(bank_df, test_size=0.2, random_state=1)

X = train.drop(columns=['Personal_Loan'])
y = train['Personal_Loan']

X_t = test.drop(columns=['Personal_Loan'])
y_t = test['Personal_Loan']

train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.375, random_state=1)

knn = KNeighborsClassifier(n_neighbors=9)
knn.fit(train_X, train_y)
pred = knn.predict(valid_X)
predprob = knn.predict_proba(valid_X)

conf_matrix = confusion_matrix(valid_y, pred)
print(conf_matrix)

accuracy = accuracy_score(valid_y, pred)
print(accuracy)



[[1311   39]
 [ 109   41]]
0.9013333333333333


In [53]:
pred = knn.predict(X_t)
predprob = knn.predict_proba(X_t)

conf_matrix = confusion_matrix(y_t, pred)
print(conf_matrix)

accuracy = accuracy_score(y_t, pred)
print(accuracy)


[[869  31]
 [ 71  29]]
0.898


In [54]:

df= pd.read_csv("D:/dmba/BostonHousing.csv")
X = df[]
train, test = train_test_split(df, test_size=0.4, random_state=1)


SyntaxError: invalid syntax (529501029.py, line 2)

In [55]:
# 7.3 a. 가장 분류 성능이 좋은 k 값을 골라야 지역적 특성에
# 민감하지 않고 빈도 수가 높은 클래스로 분류를 잘 수행할 수 있다.
# 여기에서 k는 정확도에 근거하여 4로 정했다. 

df= pd.read_csv("D:/dmba/BostonHousing.csv")

predictors = [s for s in df.columns if s != 'CAT. MEDV']
outcome = 'CAT. MEDV'

X = df[predictors]
y = df[outcome]

X_tn, X_te, y_tn, y_te = train_test_split(X, y, test_size=0.4, random_state=1)

std_scale = preprocessing.StandardScaler()
std_scale.fit(X_tn)
X_tn_std = std_scale.transform(X_tn)
X_te_std = std_scale.transform(X_te)

In [56]:
results = []
for k in range(1, 5):
    knn = KNeighborsClassifier(n_neighbors=k).fit(X_tn_std, y_tn)
    results.append({
        'k':k, 
        'accuracy_score':accuracy_score(y_te, knn.predict(X_te_std))
    })

results = pd.DataFrame(results)
print(results)
print(results[results['accuracy_score']==results['accuracy_score'].max()])

   k  accuracy_score
0  1        0.960591
1  2        0.950739
2  3        0.970443
3  4        0.970443
   k  accuracy_score
2  3        0.970443
3  4        0.970443


In [57]:
# 7.3 b. 새로운 데이터의 예측 중앙값은 19.85이다. 
# 테스트 하고자 하는 샘플 데이터와 근접한 훈련 데이터가 없을 경우,
# 예측이 부정확 하여 오류율이 높아질 수 있습니다.

from sklearn.neighbors import KNeighborsRegressor

X = df.drop(columns=['MEDV', 'CAT. MEDV'])
y = df['MEDV']

X_tn, X_te, y_tn, y_te = train_test_split(X, y, test_size=0.4, random_state=1)

std_scale = preprocessing.StandardScaler()
std_scale.fit(X_tn)
X_tn_std = std_scale.transform(X_tn)
X_te_std = std_scale.transform(X_te)


X_tn_std = pd.DataFrame(X_tn_std, columns=X_tn.columns)
X_te_std = pd.DataFrame(X_te_std, columns=X_te.columns)


knn = KNeighborsRegressor(n_neighbors=4)
knn.fit(X_tn_std, y_tn)

knn_pred = knn.predict(X_te_std)
newD = pd.DataFrame([{'CRIM':0.2, 'ZN':0, 'INDUS':7, 'CHAS':7, 'NOX':0.538, 'RM':6, 'AGE':62, 'DIS':4.7, 'RAD':4, 'TAX':307, 'PTRATIO':21, 'LSTAT':10}])

newD_df = pd.DataFrame(std_scale.fit_transform(newD), columns=newD.columns)
pred = knn.predict(newD_df)
print(pred)
distances, indices = knn.kneighbors(newD_df)
print(distances)
print(indices)

print(knn_pred)
print("테스트 세트 R^2: {:.2f}".format(knn.score(X_te_std, y_te)))

[19.85]
[[1.04988394 1.21436707 1.25278432 1.26790343]]
[[123 264 266 139]]
[28.475 24.9   19.65  21.725 19.05  18.575 28.45  17.55  21.825 26.1
 21.825 30.95  20.225 22.075 19.3   25.075  8.95  34.875 27.    15.15
 19.9   16.875 21.875 22.275 26.65   9.225 14.15  20.075 32.075 12.5
 23.6   18.225 46.2   14.175 21.9   20.85  15.4   31.3   12.375 20.425
 26.7   20.85  27.225 14.2   14.425  9.6   31.8   13.675 21.95  21.675
 21.425 21.475 25.65  21.95  12.475 22.775 10.175 21.975 17.5   34.875
 13.175 25.975 13.325 16.7   20.375 31.15  34.875 23.075 24.3   21.95
 21.075  9.2   22.925 20.975 19.575 21.35  28.05  26.425 18.4   36.075
 18.325 19.6   32.6   11.475 27.275 26.2   13.7   23.475 19.125 18.325
 23.025 40.425 14.925 21.65  10.575 20.275 22.35  22.85  34.875 19.8
 20.625 14.1   24.225 24.675 10.85  20.625 17.95  19.725 26.55  22.35
 34.225 27.4   14.65  32.8   32.075 32.6   19.75  20.55  34.775 40.425
 21.55  14.7   24.775 17.675 25.25  19.325 25.525 19.9   22.9   27.3
 21.4   21.8

In [58]:
# 8.1 a. b. 82/882

df = pd.read_csv("D:/dmba/UniversalBank.csv")
df

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,29,3,40,92697,1,1.9,3,0,0,0,0,1,0
4996,4997,30,4,15,92037,4,0.4,1,85,0,0,0,1,0
4997,4998,63,39,24,93023,2,0.3,3,0,0,0,0,0,0
4998,4999,65,40,49,90034,3,0.5,2,0,0,0,0,1,0


In [59]:
hung = df.groupby(['CreditCard', 'Personal Loan', 'Online'])['Online'].count()

hung = pd.DataFrame(hung)
hung

Online
CreditCard Personal Loan Online        
0          0             0         1300
                         1         1893
           1             0          128
                         1          209
1          0             0          527
                         1          800
           1             0           61
                         1           82

In [60]:
# 8-1 c. d. e. f. i) 143/480 ii) 291/480 iii) 480/5000 iv) 1327/4520 v)291/4520 vi)4520/5000
# 개인대출을 수락할 확률은 0.5036으로 나이브 베이즈의 확률이 더 정확한 추정이다.  
pivot1 = df.groupby(['CreditCard', 'Personal Loan'])['Personal Loan'].count()
print(pivot1)
pivot2 = df.groupby(['Online', 'Personal Loan'])['Personal Loan'].count()
print(pivot2)

s1= (143/480)* (291/480) * (480/5000)
print(s1)
s2 = (1327/4520) * (291/4520) * (4520/5000)
print(s2)
p1 = s1 / (s1+s2)
p2 = s2 / (s1+s2)
print(p1)
print(p2)

CreditCard  Personal Loan
0           0                3193
            1                 337
1           0                1327
            1                 143
Name: Personal Loan, dtype: int64
Online  Personal Loan
0       0                1827
        1                 189
1       0                2693
        1                 291
Name: Personal Loan, dtype: int64
0.01733875
0.01708659292035398
0.5036623757129943
0.4963376242870055


In [61]:
# 8.1 g. 나이브 베이즈 실행

from sklearn.naive_bayes import MultinomialNB

predictors = ['CreditCard', 'Online']
outcome = 'Personal Loan'

X = pd.get_dummies(df[predictors])
y = df['Personal Loan'].astype('category')
classes = list(y.cat.categories)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.40, random_state=1)
nb = MultinomialNB(alpha=0.01)
nb.fit(X_train, y_train)

predProb_train = nb.predict_proba(X_train)
predProb_valid = nb.predict_proba(X_valid)

y_valid_pred = nb.predict(X_valid)

df = pd.concat([pd.DataFrame({'actual':y_valid, 'predicted':y_valid_pred}), pd.DataFrame(predProb_valid, index=y_valid.index)], axis=1)
mask = ((X_valid.CreditCard == 1) & (X_valid.Online == 1))
df[mask]


,actual,predicted,0,1
932,0,0,0.904419,0.095581
1132,0,0,0.904419,0.095581
3289,0,0,0.904419,0.095581
348,1,0,0.904419,0.095581
2971,0,0,0.904419,0.095581
...,...,...,...,...
4705,1,0,0.904419,0.095581
2429,0,0,0.904419,0.095581
2684,0,0,0.904419,0.095581
3401,0,0,0.904419,0.095581


In [4]:
# 8.2 a.부상이 있다라고 답한 데이터가 더 많았으므로 부상이 있다로 예측되어야한다.

df = pd.read_csv("D:/dmba/accidentsFull.csv")
df.loc[df['MAX_SEV_IR'] == 1,'MAX_SEV_IR'] = 'YES'
df.loc[df['MAX_SEV_IR'] == 2,'MAX_SEV_IR'] = 'YES'
df.loc[df['MAX_SEV_IR'] == 0,'MAX_SEV_IR'] = 'NO'
df.loc[df['MAX_SEV_IR'] == 'YES']

C:\Users\USER\AppData\Local\Temp\ipykernel_3120\2232684275.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'YES' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['MAX_SEV_IR'] == 1,'MAX_SEV_IR'] = 'YES'


,HOUR_I_R,ALCHL_I,ALIGN_I,STRATUM_R,WRK_ZONE,WKDY_I_R,INT_HWY,LGTCON_I_R,MANCOL_I_R,PED_ACC_R,...,SUR_COND,TRAF_CON_R,TRAF_WAY,VEH_INVL,WEATHER_R,INJURY_CRASH,NO_INJ_I,PRPTYDMG_CRASH,FATALITIES,MAX_SEV_IR
0,0,2,2,1,0,1,0,3,0,0,...,4,0,3,1,1,1,1,0,0,YES
5,1,2,1,1,0,1,0,3,0,0,...,4,0,2,1,2,1,1,0,0,YES
7,1,2,1,1,0,1,0,3,0,0,...,4,0,1,1,1,1,1,0,0,YES
12,1,2,1,1,0,1,0,3,1,0,...,4,0,1,4,1,1,2,0,0,YES
14,1,2,2,1,0,1,0,3,0,0,...,4,0,1,1,1,1,1,0,0,YES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42173,0,2,1,0,1,1,0,1,0,1,...,1,1,1,1,1,1,1,0,0,YES
42174,1,2,1,0,0,1,0,1,0,1,...,1,0,1,1,1,1,2,0,0,YES
42176,1,2,1,0,0,1,0,1,0,1,...,1,0,1,1,1,1,1,0,0,YES
42177,1,2,1,0,0,1,0,1,0,1,...,1,2,1,1,1,1,1,0,0,YES


In [63]:
df.loc[df['MAX_SEV_IR'] == 'NO']

,HOUR_I_R,ALCHL_I,ALIGN_I,STRATUM_R,WRK_ZONE,WKDY_I_R,INT_HWY,LGTCON_I_R,MANCOL_I_R,PED_ACC_R,...,SUR_COND,TRAF_CON_R,TRAF_WAY,VEH_INVL,WEATHER_R,INJURY_CRASH,NO_INJ_I,PRPTYDMG_CRASH,FATALITIES,MAX_SEV_IR
1,1,2,1,0,0,1,1,3,2,0,...,4,0,3,2,2,0,0,1,0,NO
2,1,2,1,0,0,1,0,3,2,0,...,4,1,2,2,2,0,0,1,0,NO
3,1,2,1,1,0,0,0,3,2,0,...,4,1,2,2,1,0,0,1,0,NO
4,1,1,1,0,0,1,0,3,2,0,...,4,0,2,3,1,0,0,1,0,NO
6,1,2,1,0,0,1,1,3,0,0,...,4,0,2,1,2,0,0,1,0,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42175,0,2,1,0,0,1,0,1,0,0,...,1,0,1,1,1,0,0,1,0,NO
42178,0,2,1,0,1,1,0,1,0,0,...,1,2,1,1,1,0,0,1,0,NO
42180,0,2,2,0,0,1,0,1,0,0,...,1,0,1,1,1,0,0,1,0,NO
42181,1,2,1,1,0,1,0,1,0,0,...,1,0,1,1,1,0,0,1,0,NO


In [7]:
df.rename(columns={'MAX_SEV_IR':'INJURY'}, inplace=True)

In [66]:
pivot1 = df.groupby(['WEATHER_R', 'INJURY'])['INJURY'].count()
print(pivot1)
pivot2 = df.groupby(['TRAF_CON_R', 'INJURY'])['INJURY'].count()
print(pivot2)

WEATHER_R  INJURY
1          NO        17424
           YES       18736
2          NO         3297
           YES        2726
Name: INJURY, dtype: int64
TRAF_CON_R  INJURY
0           NO        13654
            YES       13327
1           NO         3910
            YES        4714
2           NO         3157
            YES        3421
Name: INJURY, dtype: int64


In [69]:
predictors = ['WEATHER_R', 'TRAF_CON_R']
outcome = 'INJURY'

train_df, valid_df = train_test_split(df, test_size=0.4, random_state=1)
print(train_df['INJURY'].value_counts()/len(train_df))
print()

for predictor in predictors:
    df = train_df[['INJURY', predictor]]
    freqTable = df.pivot_table(index='INJURY', columns=predictor, aggfunc = len)

    propTable = freqTable.apply(lambda x: x/sum(x), axis=1)
    print(propTable)
    print()

INJURY
YES    0.510372
NO     0.489628
Name: count, dtype: float64

WEATHER_R         1         2
INJURY                       
NO         0.841349  0.158651
YES        0.874119  0.125881

TRAF_CON_R         0         1         2
INJURY                                  
NO          0.659054  0.193512  0.147434
YES         0.621816  0.216149  0.162035



In [10]:
#8.2 b. iv. v.

from sklearn.naive_bayes import MultinomialNB

predictors = ['WEATHER_R', 'TRAF_CON_R']
outcome = 'INJURY'

X = pd.get_dummies(df[predictors])
y = df['INJURY'].astype('category')
classes = list(y.cat.categories)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.40, random_state=1)
nb = MultinomialNB(alpha=0.01)
nb.fit(X_train, y_train)

predProb_train = nb.predict_proba(X_train)
predProb_valid = nb.predict_proba(X_valid)

y_valid_pred = nb.predict(X_valid)

prob1 = nb.predict_proba(X_valid)[:,1]
cutoff = 0.5
y_pred_new_cutoff = [1 if i > cutoff else 0 for i in prob1]
print(predProb_valid)
print(y_pred_new_cutoff)

[[0.49967713 0.50032287]
 [0.49967713 0.50032287]
 [0.49967713 0.50032287]
 ...
 [0.49967713 0.50032287]
 [0.45511372 0.54488628]
 [0.49967713 0.50032287]]
[1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 

In [5]:
#8.2 b. i

df = pd.read_csv("D:/dmba/accidentsFull.csv")
df.rename(columns={'MAX_SEV_IR':'INJURY'}, inplace=True)
df_new = df.iloc[:12, :]
df_new


,HOUR_I_R,ALCHL_I,ALIGN_I,STRATUM_R,WRK_ZONE,WKDY_I_R,INT_HWY,LGTCON_I_R,MANCOL_I_R,PED_ACC_R,...,SUR_COND,TRAF_CON_R,TRAF_WAY,VEH_INVL,WEATHER_R,INJURY_CRASH,NO_INJ_I,PRPTYDMG_CRASH,FATALITIES,INJURY
0,0,2,2,1,0,1,0,3,0,0,...,4,0,3,1,1,1,1,0,0,1
1,1,2,1,0,0,1,1,3,2,0,...,4,0,3,2,2,0,0,1,0,0
2,1,2,1,0,0,1,0,3,2,0,...,4,1,2,2,2,0,0,1,0,0
3,1,2,1,1,0,0,0,3,2,0,...,4,1,2,2,1,0,0,1,0,0
4,1,1,1,0,0,1,0,3,2,0,...,4,0,2,3,1,0,0,1,0,0
5,1,2,1,1,0,1,0,3,0,0,...,4,0,2,1,2,1,1,0,0,1
6,1,2,1,0,0,1,1,3,0,0,...,4,0,2,1,2,0,0,1,0,0
7,1,2,1,1,0,1,0,3,0,0,...,4,0,1,1,1,1,1,0,0,1
8,1,2,1,1,0,1,0,3,0,0,...,4,0,1,1,2,0,0,1,0,0
9,0,2,1,0,0,0,0,3,0,0,...,4,0,1,1,2,0,0,1,0,0


In [8]:
pivot1 = df_new.groupby(['WEATHER_R', 'INJURY'])['INJURY'].count()
pivot1 = pd.DataFrame(pivot1)
print(pivot1)

pivot2 = df_new.groupby(['TRAF_CON_R', 'INJURY'])['INJURY'].count()
pivot2 = pd.DataFrame(pivot2)
print(pivot2)

                  INJURY
WEATHER_R INJURY        
1         0            3
          1            2
2         0            6
          1            1
                   INJURY
TRAF_CON_R INJURY        
0          0            6
           1            3
1          0            2
2          0            1


In [10]:
#8.2 b. ii, iii. 
# p_1_1_0 = 0.571429 *  0.666667 * 0.75
# p_1_2_0 = 0.571429 *  0.666667 * 0.25


predictors = ['WEATHER_R', 'TRAF_CON_R']
outcome = 'INJURY'

train_df, valid_df = train_test_split(df_new, test_size=0.4, random_state=1)
print(train_df['INJURY'].value_counts()/len(train_df))
print()

for predictor in predictors:
    df = train_df[['INJURY', predictor]]
    freqTable = df.pivot_table(index='INJURY', columns=predictor, aggfunc = len)

    propTable = freqTable.apply(lambda x: x/sum(x), axis=1)
    print(propTable)
    print()

INJURY
0    0.571429
1    0.428571
Name: count, dtype: float64

WEATHER_R         1         2
INJURY                       
0          0.250000  0.750000
1          0.666667  0.333333

TRAF_CON_R     0     2
INJURY                
0           0.75  0.25
1            NaN   NaN

